## Imports

In [1]:
import pandas as pd
import numpy as np
import torch
import json

from tqdm import tqdm
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer
from os import walk
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_curve, auc, brier_score_loss

c:\Users\boyan.bogdanov\.conda\envs\llm-detect\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
ROOT_DATA_RAW = '../../data/raw'
HUMAN_JSON_FILE_NAME = 'human.jsonl'
HUMAN_JSON_PATH = f'{ROOT_DATA_RAW}/{HUMAN_JSON_FILE_NAME}'
MODELS_JSON_FOLDER_PATH = f'{ROOT_DATA_RAW}/machines'

In [3]:
BATCH_SIZE = 32
LSTM_UNITS = 256
LSTM_LAYERS = 5
EMBEDDING_SIZE = 512

In [4]:
TEST_SET_FRACTION = 0.3

In [5]:
def extract_topic(id):
    try:
        dash_indices = [i for i, char in enumerate(id) if char == '-']
        second_last_dash_index = dash_indices[-2]
        last_slash_index = id.rindex('/')
        
        # Extract the text between the second-to-last "-" and the last "/"
        result = id[second_last_dash_index + 1:last_slash_index]
        return result
    except ValueError:
        # In case "-" or "/" are not found, return None
        return None

In [6]:
df = pd.read_json(path_or_buf=HUMAN_JSON_PATH, lines=True)
df['text_index'] = df.index
df['is_llm'] = 0
df['topic'] = df['id'].apply(extract_topic)

In [7]:
dir_path, dir_names, file_names = next(walk(MODELS_JSON_FOLDER_PATH))

for file_name in file_names:
    temp_df = pd.read_json(path_or_buf=f'{MODELS_JSON_FOLDER_PATH}/{file_name}', lines=True)
    temp_df['text_index'] = temp_df.index
    temp_df['is_llm'] = 1
    temp_df['topic'] = temp_df['id'].apply(extract_topic)

    df = pd.concat([df, temp_df], ignore_index=True)

df.drop(labels=['id'], inplace=True, axis='columns')

In [8]:
df.tail()

,text,text_index,is_llm,topic
15213,'The Disappearance of Gabby Petito' – A Compre...,1082,1,wyominggabbypetito
15214,"Utah State Police Search for Gabby Petito, Tra...",1083,1,wyominggabbypetito
15215,McKenna's Lost Friend: Debunking the Evidence ...,1084,1,wyominggabbypetito
15216,"""Gunshots Found in Florida Nature Preserve: A ...",1085,1,wyominggabbypetito
15217,A Very Kind and Sweet Woman in Long Island Sho...,1086,1,wyominggabbypetito


### Tokenize

In [9]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
df['tokenized_text'] = tokenizer(list(df['text'].to_list()))['input_ids']

Token indices sequence length is longer than the specified maximum sequence length for this model (843 > 512). Running this sequence through the model will result in indexing errors


,text,text_index,is_llm,topic,tokenized_text
0,Inaugural Address by President Joseph R. Biden...,0,0,bideninauguration,"[101, 1130, 3984, 13830, 4412, 24930, 18380, 1..."
1,Fact check: Biden inauguration impacted by pan...,1,0,bideninauguration,"[101, 143, 11179, 4031, 131, 139, 26859, 20105..."
2,Highlights from Joe Biden's 2021 inauguration\...,2,0,bideninauguration,"[101, 1693, 13231, 1121, 2658, 139, 26859, 112..."
3,"Biden takes the helm, appeals for unity to tak...",3,0,bideninauguration,"[101, 139, 26859, 2274, 1103, 22778, 117, 1599..."
4,'The Hill We Climb': Read Amanda Gorman's inau...,4,0,bideninauguration,"[101, 112, 1109, 2404, 1284, 140, 24891, 1830,..."
...,...,...,...,...,...
15174,'Massive Impact of Winter Storm in Houston'\n\...,1043,1,winterstormtexas,"[101, 112, 8718, 2109, 13788, 1104, 4591, 7007..."
15175,Ken Paxton's Accusations of Shifting Weather E...,1044,1,winterstormtexas,"[101, 5928, 19585, 14933, 112, 188, 138, 19515..."
15176,Two Fires Have Emerged in Austin After Extreme...,1045,1,winterstormtexas,"[101, 1960, 4266, 1116, 4373, 18653, 1200, 366..."
15177,Sugar Land Fire: A Story Conquered by Children...,1046,1,winterstormtexas,"[101, 10619, 4026, 4266, 131, 138, 5145, 16752..."


## Model

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [12]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, layers_num, device, output_size=1, dropout=0):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.layers_num = layers_num
        self.output_size= output_size
        self.dropout = dropout
        self.device = device

        self.embed = nn.Embedding(self.vocab_size, self.embedding_size, device=self.device)

        self.lstm = nn.LSTM(
            input_size=self.embedding_size,
            hidden_size=self.hidden_size,
            num_layers=self.layers_num,
            batch_first=True,
            dropout=self.dropout,
            device=self.device
        )

        self.fc = nn.Linear(
            self.hidden_size,
            self.output_size
        )

    def forward(self, X, lengths):
        embeddings = self.embed(X)

        seq_output, (h_n, c_n) = self.lstm(embeddings)

        out = seq_output.sum(dim=1).div(lengths.float().unsqueeze(dim=1))
        logits = self.fc(out)
        return logits

## Dataset

In [13]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.size

    def __getitem__(self, index):
        return (
            self.X.iloc[index],
            self.y.iloc[index]
        )

In [14]:
def collate_fn(batch):
  # We want to sort the batch by seq length,
  # in order to make the computation more efficient
  batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

  inputs = [torch.LongTensor(x[0]).to(device) for x in batch]
  padded_input = nn.utils.rnn.pad_sequence(inputs, batch_first=True)

  lengths = torch.LongTensor([len(x[0]) for x in batch]).to(device)

  y = torch.FloatTensor(np.array([x[1] for x in batch])).reshape(-1, 1).to(device)

  return padded_input, lengths, y

## Train and test functions

In [15]:
def calculate_accuracy(y_true, y_hat):
    correct_pred = torch.eq(torch.sigmoid(y_hat).round(), y_true).sum().item()
    return (correct_pred / len(y_hat)) * 100

def calculate_f1(y_true, y_hat):
    y_pred = torch.sigmoid(y_hat).round()
    return f1_score(y_true, y_pred)

def calculate_brier(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)
    return brier_score_loss(y_true, y_prob)

def calculate_auc(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)

    false_positive_rates, true_positive_rates, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(false_positive_rates, true_positive_rates)

    return roc_auc, false_positive_rates, true_positive_rates

def train_step(model, dataloader, loss_fn, optimizer, device):

    model.train()

    train_loss, train_acc = 0, 0
    steps = 0

    for X, lengths, y in dataloader:
        X, y = X.to(device), y.to(device)

        y_hat = model(X, lengths)

        loss = loss_fn(y_hat, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc += calculate_accuracy(y_true=y, y_hat=y_hat)
        steps += 1

    return train_loss / steps, train_acc / steps


def test_step(model, dataloader, loss_fn, device):

    model.eval()

    all_y_true = []
    all_y_hat = []

    test_loss = 0
    steps = 0

    with torch.inference_mode():
        for X, lengths, y in dataloader:

            X, y = X.to(device), y.to(device)

            y_hat = model(X, lengths)

            all_y_true.extend(y)
            all_y_hat.extend(y_hat)

            loss = loss_fn(y_hat, y)
            test_loss += loss.item()

            steps += 1

        all_y_true = torch.FloatTensor(all_y_true)
        all_y_hat = torch.FloatTensor(all_y_hat)

        test_accuracy = calculate_accuracy(all_y_true, all_y_hat)
        test_f1 = calculate_f1(all_y_true, all_y_hat)
        test_brier = calculate_brier(all_y_true, all_y_hat)
        test_auc_tuple = calculate_auc(all_y_true, all_y_hat)

    return test_loss / steps, test_accuracy, test_f1, test_brier, test_auc_tuple

def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          epochs,
          device):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
        "test_f1": [],
        "test_brier": [],
        "test_auc_tuple": []
    }

    model.to(device)

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        end_time = timer()

        test_loss, test_acc, test_f1, test_brier, test_auc_tuple = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        results["test_f1"].append(test_f1)
        results["test_brier"].append(test_brier),
        results["test_auc_tuple"].append(test_auc_tuple)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"test_f1: {test_f1:.4f} | "
            f"test_brier: {test_brier:.4f} | "
            f"time: {(end_time-start_time):.4f}"
        )

    return results

## Train on each LLM dataset separately and test against all others

In [21]:
def test_against_all(model, topic, df, loss_fn, device):
    all_results = []

    print(topic)

    for other_topic in df['topic'].unique():
        if other_topic == topic:
            continue

        other_topics_df = df.loc[df['topic'] == other_topic]
        print(other_topic, other_topics_df.shape[0])
        test_dataset = TextDataset(other_topics_df['tokenized_text'], other_topics_df['is_llm'])
        test_dataloader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            drop_last=False,
            collate_fn=collate_fn
        )

        results = test_step(
            model,
            test_dataloader,
            loss_fn,
            device
        )

        all_results.append({
            topic: results
        })

    return all_results

In [17]:
human_df = df.loc[df['is_llm'] == 0]
human_train_df, human_test_df = train_test_split(human_df, test_size=TEST_SET_FRACTION, random_state=69)

In [22]:
final_results = []

for topic in df['topic'].unique():
    topic_df = df.loc[df['topic'] == topic]
    topic_train_df, topic_test_df = train_test_split(topic_df, test_size=TEST_SET_FRACTION, random_state=69)

    train_dataset = TextDataset(topic_train_df['tokenized_text'], topic_train_df['is_llm'])
    test_dataset = TextDataset(topic_test_df['tokenized_text'], topic_test_df['is_llm'])

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=False,
        collate_fn=collate_fn
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=False,
        collate_fn=collate_fn
    )

    model = RNN(tokenizer.vocab_size, EMBEDDING_SIZE, LSTM_UNITS, LSTM_LAYERS, device, dropout=0.6).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00008)
    loss_fn = nn.BCEWithLogitsLoss()

    print(f'Training model for {topic}...')

    current_results = train(
        model,
        train_dataloader,
        test_dataloader,
        optimizer,
        loss_fn,
        epochs=4,
        device=device
    )    

    print(f'Finished training model for {topic}')

    print(f'Testing against all for {topic}...')
    
    results_against_all = test_against_all(
        model,
        topic,
        df,
        loss_fn,
        device
    )

    final_results.append({
        'results_against_itself': current_results,
        'results_against_all': results_against_all
    })

    print(f'Finished testing against all for {topic}')

Training model for bideninauguration...


 25%|██▌       | 1/4 [00:53<02:41, 53.68s/it]

Epoch: 1 | train_loss: 0.6205 | train_acc: 93.9732 | test_loss: 0.5578 | test_acc: 91.1602 | test_f1: 0.9538 | test_brier: 0.1863 | time: 49.4866


 50%|█████     | 2/4 [01:48<01:48, 54.32s/it]

Epoch: 2 | train_loss: 0.4640 | train_acc: 92.7679 | test_loss: 0.3497 | test_acc: 91.1602 | test_f1: 0.9538 | test_brier: 0.1016 | time: 49.7563


...

 75%|███████▌  | 3/4 [03:52<01:17, 77.12s/it]

Epoch: 3 | train_loss: 0.3086 | train_acc: 94.4965 | test_loss: 0.3506 | test_acc: 89.0244 | test_f1: 0.9419 | test_brier: 0.0967 | time: 71.5841


100%|██████████| 4/4 [05:12<00:00, 78.04s/it]

Epoch: 4 | train_loss: 0.2413 | train_acc: 94.4792 | test_loss: 0.3721 | test_acc: 89.0244 | test_f1: 0.9419 | test_brier: 0.1001 | time: 74.9245
Finished training model for wyominggabbypetito
Testing against all for wyominggabbypetito...
wyominggabbypetito
bideninauguration 602


capitolriot 630
citibank500millionmistake 210
cnnchriscuomo 644
colinpowelldead 560
colonialpipelinehack 756
covid19 854
evergreensuezcanal 644
facebookoutage 644
georgefloydderekchauvin 490
harryandmeghan 714
hurricaneida 686
kabulairportattack 812
kamalaharrisvicepresident 588
kylerittenhousenotguilty 756
michiganhighschoolshooting 728
rustmoviesetshooting 616
stimuluscheck 672
tigerwoodsaccident 728
tombradysuperbowl 630
trumpimpeachment 574
twitterbanstrump 728
winterstormtexas 406
Finished testing against all for wyominggabbypetito


In [134]:
results_copy = final_results[:]

topics = df['topic'].unique().tolist()
topics

['bideninauguration',
 'capitolriot',
 'citibank500millionmistake',
 'cnnchriscuomo',
 'colinpowelldead',
 'colonialpipelinehack',
 'covid19',
 'evergreensuezcanal',
 'facebookoutage',
 'georgefloydderekchauvin',
 'harryandmeghan',
 'hurricaneida',
 'kabulairportattack',
 'kamalaharrisvicepresident',
 'kylerittenhousenotguilty',
 'michiganhighschoolshooting',
 'rustmoviesetshooting',
 'stimuluscheck',
 'tigerwoodsaccident',
 'tombradysuperbowl',
 'trumpimpeachment',
 'twitterbanstrump',
 'winterstormtexas',
 'wyominggabbypetito']

In [131]:
for row in results_copy:
    iter = row['results_against_all'][:]
    for i, x in enumerate(row['results_against_all'][:][0].keys()):
        iter[0][topics[i]] = iter[0][x]
    row['results_against_all'][0] = iter    

RuntimeError: dictionary changed size during iteration

In [111]:
results_copy[0]['results_against_all']

[{'bideninauguration': (0.3489207586756459,
   92.85714285714286,
   0.9629629629629629,
   0.0793907960174134)},
 {'bideninauguration': (0.32285482196935583,
   92.85714285714286,
   0.9629629629629629,
   0.0775852310544822)},
 {'bideninauguration': (0.30391925892659594,
   92.85714285714286,
   0.9629629629629629,
   0.0756026320439937)},
 {'bideninauguration': (0.28682881221175194,
   92.85714285714286,
   0.9629629629629629,
   0.07332814442912951)},
 {'bideninauguration': (0.3266417782591737,
   92.85714285714286,
   0.9629629629629629,
   0.0764303416747718)},
 {'bideninauguration': (0.3167598830028014,
   92.85714285714286,
   0.9629629629629629,
   0.07653539716091126)},
 {'bideninauguration': (0.40827500354498625,
   92.85714285714286,
   0.9629629629629629,
   0.08376447079317795)},
 {'bideninauguration': (0.3796791381349689,
   92.85714285714286,
   0.9629629629629629,
   0.0777435294284679)},
 {'bideninauguration': (0.3062702782141666,
   92.85714285714286,
   0.9629629629

In [108]:
for r in results_copy:
    for item in r['results_against_all']:
        print(item)


ValueError: list.remove(x): x not in list

In [135]:
with open('topic-data.json', 'w') as f:
    json.dump(final_results, f)